<a href="https://colab.research.google.com/github/Judy-Choi/Tensorflow_Certificate/blob/main/11_BBC_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal Metric - val_loss 0.18

# Solution - 0.17959


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, GlobalAveragePooling1D
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


bbcfile = "/content/drive/MyDrive/CEO_AI_Coding/01-CEO/CEO_11기/deep_learning/4. BBC/bbc-text.csv"
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = ""
training_portion = .8


sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an",
    "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below",
    "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
    "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd",
    "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", 
    "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's",
    "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", 
    "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same",
    "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", 
    "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", 
    "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", 
    "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what",
    "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
    "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", 
    "yours", "yourself", "yourselves" ]

with open(bbcfile, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

model = tf.keras.Sequential([
   Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(6, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
    save_weights_only=True, 
    save_best_only=True, 
    monitor='val_loss',
    verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
num_epochs = 100
history = model.fit(train_padded, training_label_seq, 
    epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), 
    callbacks=[checkpoint, early_stopping],
    verbose=1)

model.save("mymodel.h5")

Epoch 1/100
48/56 [========================>.....] - ETA: 0s - loss: 1.7698 - accuracy: 0.2415
Epoch 1: val_loss improved from inf to 1.73697, saving model to my_checkpoint.ckpt
56/56 [==============================] - 1s 7ms/step - loss: 1.7654 - accuracy: 0.2438 - val_loss: 1.7370 - val_accuracy: 0.2292
Epoch 2/100
46/56 [=======================>......] - ETA: 0s - loss: 1.7012 - accuracy: 0.2303
Epoch 2: val_loss improved from 1.73697 to 1.65555, saving model to my_checkpoint.ckpt
56/56 [==============================] - 0s 4ms/step - loss: 1.6930 - accuracy: 0.2309 - val_loss: 1.6556 - val_accuracy: 0.2292
Epoch 3/100
44/56 [======================>.......] - ETA: 0s - loss: 1.6153 - accuracy: 0.2393
Epoch 3: val_loss improved from 1.65555 to 1.57349, saving model to my_checkpoint.ckpt
56/56 [==============================] - 0s 4ms/step - loss: 1.6079 - accuracy: 0.2539 - val_loss: 1.5735 - val_accuracy: 0.3079
Epoch 4/100
52/56 [==========================>...] - ETA: 0s - loss: 1.

# Homework 1 : Bidirectional LSTM - 0.50866

In [11]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, GlobalAveragePooling1D, Dropout, BatchNormalization, Flatten
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


bbcfile = "/content/drive/MyDrive/TF_WISET/bbc-text.csv"
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = ""
training_portion = .8


sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an",
    "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below",
    "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
    "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd",
    "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", 
    "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's",
    "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", 
    "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same",
    "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", 
    "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", 
    "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", 
    "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what",
    "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
    "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", 
    "yours", "yourself", "yourselves" ]

with open(bbcfile, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))


# Define the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    BatchNormalization(),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    BatchNormalization(),
    Bidirectional(LSTM(64)),
    Dropout(0.2),
    BatchNormalization(),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(6, activation='softmax')
])


model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
    save_weights_only=True, 
    save_best_only=True, 
    monitor='val_loss',
    verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
num_epochs = 100

history = model.fit(train_padded, training_label_seq, 
    epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), 
    callbacks=[checkpoint, early_stopping],
    verbose=1)

model.save("mymodel.h5")

Epoch 1/100
56/56 [==============================] - ETA: 0s - loss: 1.9968 - accuracy: 0.2730
Epoch 1: val_loss improved from inf to 1.69463, saving model to my_checkpoint.ckpt
56/56 [==============================] - 21s 157ms/step - loss: 1.9968 - accuracy: 0.2730 - val_loss: 1.6946 - val_accuracy: 0.2472
Epoch 2/100
56/56 [==============================] - ETA: 0s - loss: 1.0991 - accuracy: 0.5815
Epoch 2: val_loss improved from 1.69463 to 1.63111, saving model to my_checkpoint.ckpt
56/56 [==============================] - 4s 76ms/step - loss: 1.0991 - accuracy: 0.5815 - val_loss: 1.6311 - val_accuracy: 0.2382
Epoch 3/100
56/56 [==============================] - ETA: 0s - loss: 0.5411 - accuracy: 0.8292
Epoch 3: val_loss improved from 1.63111 to 1.57497, saving model to my_checkpoint.ckpt
56/56 [==============================] - 3s 60ms/step - loss: 0.5411 - accuracy: 0.8292 - val_loss: 1.5750 - val_accuracy: 0.5056
Epoch 4/100
56/56 [==============================] - ETA: 0s - los

# Homework 2 : ChatGPT - 0.19951

다음 코드를 최적화해 줘,
단, 다음 조건을 적용해 줘
Use dropout
Use batch normalization
Use checkpoint with save best only, and save weight only and verbose = 1
Use early stopping with patience = 10
Use reduce learning rate on plateau
set verbose = 1
Set epochs = 50
do not use data augmentation
Use other methods to make val loss value lower.

ChatGPT Answer :

Here's what I've done:

Imported the necessary layers for Dropout and BatchNormalization from keras.layers.
Added a BatchNormalization() layer after the first Dense layer to normalize the activations and improve training speed.
Added a Dropout(0.5) layer after the BatchNormalization layer to prevent overfitting.
Changed the optimizer to Adam and set the learning rate to 0.001.
Added a ReduceLROnPlateau callback to reduce the learning rate if the validation loss plateaus.
Reduced the number of epochs to 50 to prevent overfitting.
Added verbose=1 to print progress during training.
These changes should help improve the performance of your model and prevent overfitting.

In [14]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


bbcfile = "/content/drive/MyDrive/TF_WISET/bbc-text.csv"
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = ""
training_portion = .8


sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an",
    "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below",
    "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
    "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd",
    "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", 
    "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's",
    "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", 
    "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same",
    "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", 
    "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", 
    "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", 
    "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what",
    "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom",
    "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", 
    "yours", "yourself", "yourselves" ]

with open(bbcfile, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
        sentences.append(sentence)

train_size = int(len(sentences) * training_portion)

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

model = tf.keras.Sequential([
   Embedding(vocab_size, embedding_dim, input_length=max_length),
   GlobalAveragePooling1D(),
   Dense(24, activation='relu'),
   BatchNormalization(),
   Dropout(0.5),
   Dense(6, activation='softmax')
])

optimizer = Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
    save_weights_only=True, 
    save_best_only=True, 
    monitor='val_loss',
    verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
    save_weights_only=True, 
    save_best_only=True, 
    monitor='val_loss',
    verbose=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

num_epochs = 50

history = model.fit(train_padded, training_label_seq, 
    epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), 
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1)

model.save("mymodel.h5")

Epoch 1/50
56/56 [==============================] - ETA: 0s - loss: 1.5788 - accuracy: 0.5230
Epoch 1: val_loss improved from inf to 1.72373, saving model to my_checkpoint.ckpt
56/56 [==============================] - 6s 84ms/step - loss: 1.5788 - accuracy: 0.5230 - val_loss: 1.7237 - val_accuracy: 0.3303 - lr: 0.0010
Epoch 2/50
56/56 [==============================] - ETA: 0s - loss: 1.0258 - accuracy: 0.7983
Epoch 2: val_loss improved from 1.72373 to 1.62707, saving model to my_checkpoint.ckpt
56/56 [==============================] - 2s 41ms/step - loss: 1.0258 - accuracy: 0.7983 - val_loss: 1.6271 - val_accuracy: 0.5303 - lr: 0.0010
Epoch 3/50
56/56 [==============================] - ETA: 0s - loss: 0.6449 - accuracy: 0.8770
Epoch 3: val_loss improved from 1.62707 to 1.50849, saving model to my_checkpoint.ckpt
56/56 [==============================] - 3s 50ms/step - loss: 0.6449 - accuracy: 0.8770 - val_loss: 1.5085 - val_accuracy: 0.7798 - lr: 0.0010
Epoch 4/50
56/56 [==============